### Deliverable 3: Optimization
Attempt 3: Change activation function

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import os 
from tensorflow.keras.callbacks import ModelCheckpoint

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
#application_df.head()

In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)
#application_df.head()

In [3]:
# Determine the number of unique values in each column.
#application_df.nunique()

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_type_counts = application_df.APPLICATION_TYPE.value_counts()
#application_type_counts

In [5]:
# Visualize the value counts of APPLICATION_TYPE
#application_type_counts.plot.density()

In [6]:
# Determine which values to replace if counts are less than ...?
replace_application_types = list(application_type_counts[application_type_counts < 500].index)

# Replace in dataframe
for app in replace_application_types:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df.CLASSIFICATION.value_counts()
#classification_counts

In [8]:
# Visualize the value counts of CLASSIFICATION
#classification_counts.plot.density()

In [9]:
# Determine which values to replace if counts are less than ..?
replace_classification = list(classification_counts[classification_counts < 1000].index)

# Replace in dataframe
for cls in replace_classification:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [10]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
#application_cat

In [11]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [12]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True,right_index=True).drop(application_cat,1)

In [13]:
application_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34295,1,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34296,1,5000,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34297,1,5000,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [14]:
# Split our preprocessed data into our features and target arrays
y = application_df.IS_SUCCESSFUL
X = application_df.drop(columns=["IS_SUCCESSFUL"])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [15]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Deliverable 2: Compile, Train and Evaluate the Model

In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])

number_input_features

43

In [17]:
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 80
hidden_nodes_layer3 = 50

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               4400      
_________________________________________________________________
dense_1 (Dense)              (None, 80)                8080      
_________________________________________________________________
dense_2 (Dense)              (None, 50)                4050      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 51        
Total params: 16,581
Trainable params: 16,581
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create callback
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=250)

In [19]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50,callbacks=[cp_callback])

Epoch 1/50
243/804 [========>.....................] - ETA: 1s - loss: 0.5991 - accuracy: 0.6975
Epoch 00001: saving model to checkpoints\weights.01.hdf5
492/804 [=================>............] - ETA: 0s - loss: 0.5900 - accuracy: 0.7052
Epoch 00001: saving model to checkpoints\weights.01.hdf5
740/804 [==========================>...] - ETA: 0s - loss: 0.5844 - accuracy: 0.7095
Epoch 00001: saving model to checkpoints\weights.01.hdf5
804/804 [==============================] - 3s 2ms/step - loss: 0.5833 - accuracy: 0.7104
Epoch 2/50
184/804 [=====>........................] - ETA: 1s - loss: 0.5436 - accuracy: 0.7412
Epoch 00002: saving model to checkpoints\weights.02.hdf5
438/804 [===============>..............] - ETA: 0s - loss: 0.5511 - accuracy: 0.7347
Epoch 00002: saving model to checkpoints\weights.02.hdf5
689/804 [========================>.....] - ETA: 0s - loss: 0.5544 - accuracy: 0.7318
Epoch 00002: saving model to checkpoints\weights.02.hdf5
804/804 [============================

742/804 [==========================>...] - ETA: 0s - loss: 0.5460 - accuracy: 0.7319
Epoch 00015: saving model to checkpoints\weights.15.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5456 - accuracy: 0.7323
Epoch 16/50
168/804 [=====>........................] - ETA: 1s - loss: 0.5419 - accuracy: 0.7341 ETA: 1s - loss: 0.5491 - accura
Epoch 00016: saving model to checkpoints\weights.16.hdf5
433/804 [===============>..............] - ETA: 0s - loss: 0.5423 - accuracy: 0.7353
Epoch 00016: saving model to checkpoints\weights.16.hdf5
673/804 [========================>.....] - ETA: 0s - loss: 0.5426 - accuracy: 0.7348
Epoch 00016: saving model to checkpoints\weights.16.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5425 - accuracy: 0.7350
Epoch 17/50
121/804 [===>..........................] - ETA: 1s - loss: 0.5292 - accuracy: 0.7476
Epoch 00017: saving model to checkpoints\weights.17.hdf5
375/804 [============>.................] - ETA: 0s - los

Epoch 00030: saving model to checkpoints\weights.30.hdf5
426/804 [==============>...............] - ETA: 0s - loss: 0.5428 - accuracy: 0.7349
Epoch 00030: saving model to checkpoints\weights.30.hdf5
667/804 [=======================>......] - ETA: 0s - loss: 0.5417 - accuracy: 0.7357
Epoch 00030: saving model to checkpoints\weights.30.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5408 - accuracy: 0.7363
Epoch 31/50
110/804 [===>..........................] - ETA: 1s - loss: 0.5513 - accuracy: 0.7274
Epoch 00031: saving model to checkpoints\weights.31.hdf5
366/804 [============>.................] - ETA: 1s - loss: 0.5388 - accuracy: 0.7351
Epoch 00031: saving model to checkpoints\weights.31.hdf5
613/804 [=====================>........] - ETA: 0s - loss: 0.5377 - accuracy: 0.7354
Epoch 00031: saving model to checkpoints\weights.31.hdf5
804/804 [==============================] - 2s 3ms/step - loss: 0.5372 - accuracy: 0.7360
Epoch 32/50
 74/804 [=>....................

804/804 [==============================] - 2s 2ms/step - loss: 0.5343 - accuracy: 0.7426
Epoch 45/50
112/804 [===>..........................] - ETA: 1s - loss: 0.5315 - accuracy: 0.7379
Epoch 00045: saving model to checkpoints\weights.45.hdf5
363/804 [============>.................] - ETA: 1s - loss: 0.5355 - accuracy: 0.7376
Epoch 00045: saving model to checkpoints\weights.45.hdf5
610/804 [=====================>........] - ETA: 0s - loss: 0.5357 - accuracy: 0.7383
Epoch 00045: saving model to checkpoints\weights.45.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5351 - accuracy: 0.7391
Epoch 46/50
 64/804 [=>............................] - ETA: 1s - loss: 0.5316 - accuracy: 0.7472
Epoch 00046: saving model to checkpoints\weights.46.hdf5
306/804 [==========>...................] - ETA: 1s - loss: 0.5345 - accuracy: 0.7376
Epoch 00046: saving model to checkpoints\weights.46.hdf5
551/804 [===================>..........] - ETA: 0s - loss: 0.5347 - accuracy: 0.7371
Epo

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5566 - accuracy: 0.7291
Loss: 0.556590735912323, Accuracy: 0.7290962338447571


In [21]:
# Export your results to an HDF5 file
nn.save('AlphabetSoupCharity_attempt3.h5')